# 역할

대문자로 뽑았을 때, 빈도수 2개 이상인 애들, 총 데이터 10개 이상인 애들만 남겨서 1000개 깔끔하게 뽑아보기
최종 결과물은 대문자 소문자 섞인 애들로?? -> 이거는 케이스마다 나눠서

In [1]:
pwd

'/home/gyubin/Knowledge_Editing_Dataset/analysis/Similarity'

In [2]:
from collections import Counter
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import re



In [3]:
df = pd.read_excel("../Preprocessing/well_df_19124.xlsx")
answer = df.copy()
df.head()

,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,sbj_views,wiki_subject
0,0,Danielle Darrieux,Q234149,"voice,European Film Award for Best Actress,Por...",20,French,Q150,"Province of Namur,present tense,present partic...",72,English,Q1860,"Malta,Bahrain,Israel,Sierra Leone,United Arab ...",149,222655,Danielle Darrieux
1,1,Edwin of Northumbria,Q348955,"male,Hatfield Chase,Cwenburh,English,saint,hum...",11,Christianity,Q5043,"christianisme,kristendom,cristianesimo,christe...",17,Islam,Q432,"Arabian mythology,Battle of Badr,Muslim world,...",68,176159,Edwin of Northumbria
2,2,Toko Yasuda,Q7813654,"rock music,Blonde Redhead,plvsvltramusic,music...",12,guitar,Q6607,"mahogany wood,Instrument of the Year,tgu,Gitaa...",28,piano,Q5994,"Klavier,klaver,kpf,A,pianist,Bartolomeo Cristo...",6,74431,Toko Yasuda
3,3,Autonomous University of Madrid,Q788091,"Coalition for Advancing Research Assessment,Eu...",8,Spain,Q29,"Romani people,Valencian Community,Portugal,Gal...",153,Sweden,Q34,"Partnership for Peace,International Centre for...",137,87459,Autonomous University of Madrid
4,5,Thomas Joannes Stieltjes,Q510916,"Jean Gaston Darboux,cemetery of Terre Cabade,T...",18,Dutch,Q7411,"present tense,Low Franconian,nl,nld,dbr,dut,dl...",42,English,Q1860,"Malta,Bahrain,Vanuatu,Israel,Sierra Leone,Unit...",160,29584,Thomas Joannes Stieltjes


In [6]:
x = 0
for i in range(len(df)):
    try:
        a = df.loc[i, 'sbj_one_hop'].split(',')
        if (len(a) != len(set(a))):
            x+=1
    except:
        pass
print(x)

471


In [ ]:
def sort_one_hop(df, column, counter, idx, rank=3, min_count=2):
    new_dict = dict()
    word_list = df.loc[idx, column].split(',')
    for word in word_list:
        if counter[word] >= min_count:  # 빈도수가 min_count 이상인 경우에만 추가
            new_dict[word] = counter[word]
    new_dict = dict(sorted(new_dict.items(), key=lambda item: item[1], reverse = False))
    new_list = list(new_dict.keys())
    return new_list

In [3]:
list20 = []
k = 10
for i in range(len(df)):
    if df.loc[i, 'sbj_hop_num']>=k and df.loc[i, 'obj_true_hop_num']>=k and df.loc[i, 'obj_new_hop_num']>=k:
        list20.append(i)
df = df.iloc[list20]
df = df.reset_index(drop = True)
print(len(df))

7480


In [4]:
def filter_capitalized_words(words, opt = 1):
    ans = [word for word in words if word[0].isupper()]
    num = len(ans)
    if (opt == 0):
        return num, ','.join(words)
    else:
        return num, ','.join(ans)

for i in range(len(df)):
    df.loc[i, 'sbj_hop_num'], df.loc[i, 'sbj_one_hop'] = filter_capitalized_words(df.loc[i, 'sbj_one_hop'].split(','))
    df.loc[i, 'obj_true_hop_num'], df.loc[i, 'obj_one_hop'] = filter_capitalized_words(df.loc[i, 'obj_one_hop'].split(','))
    df.loc[i, 'obj_new_hop_num'], df.loc[i, 'obj_new_one_hop'] = filter_capitalized_words(df.loc[i, 'obj_new_one_hop'].split(','))

In [5]:
list20 = []
for i in range(len(df)):
    if df.loc[i, 'sbj_hop_num']>=k and df.loc[i, 'obj_true_hop_num']>=k and df.loc[i, 'obj_new_hop_num']>=k:
        list20.append(i)
df = df.iloc[list20]
df = df.reset_index(drop = True)
print(len(df))

3097


In [ ]:
frequency_sbj = []
for i in range(len(df)):
    try:
        for word in (df.loc[i, 'sbj_one_hop'].split(',')):
            frequency_sbj.append(word)
    except:
        pass
counter_sbj = Counter(frequency_sbj)

# obj true counter
frequency_obj = []
for i in range(len(df)):
    try:
        for word in (df.loc[i, 'obj_one_hop'].split(',')):
            frequency_obj.append(word)
    except:
        pass
counter_obj = Counter(frequency_obj)
 
# obj new counter
frequency_obj_new = []
for i in range(len(df)):
    try:
        for word in (df.loc[i, 'obj_new_one_hop'].split(',')):
            frequency_obj_new.append(word)
    except:
        pass
counter_obj_new = Counter(frequency_obj_new)

In [ ]:

copy_df = df.copy()

empty_df = pd.DataFrame(columns=df.columns)

for i in tqdm(range(len(df))):
    idx = i
    rank = 10
    try:
        z= sort_one_hop(df, 'sbj_one_hop', counter_sbj, idx, rank)
        x= sort_one_hop(df, 'obj_one_hop', counter_obj, idx, rank)
        c =sort_one_hop(df, 'obj_new_one_hop', counter_obj_new, idx, rank)
        if len(z)+len(x)+len(c) == rank*3:
            copy_df.loc[i, 'sbj_one_hop'] = ','.join(z)
            copy_df.loc[i, 'obj_one_hop'] = ','.join(x)
            copy_df.loc[i, 'obj_new_one_hop'] = ','.join(c)
            empty_df = pd.concat([empty_df, copy_df.iloc[[i]]], ignore_index=True)

            
    except:
        print('error')

print(len(empty_df))
print(len(df))

100%|██████████| 3097/3097 [00:01<00:00, 1696.51it/s]

1700
3097


In [ ]:
def get_random_integers(start, end, count=25):
    
    return random.sample(range(start, end), count)
best_list = get_random_integers(0, len(empty_df), count = 1000)
best_list.sort()
empty_df = empty_df.iloc[best_list]
empty_df = empty_df.reset_index(drop = True)

In [10]:
for i in range(len(empty_df)):
    empty_df.loc[i, 'sbj_hop_num'], empty_df.loc[i, 'sbj_one_hop'] = filter_capitalized_words(empty_df.loc[i, 'sbj_one_hop'].split(','), 0)
    empty_df.loc[i, 'obj_true_hop_num'], empty_df.loc[i, 'obj_one_hop'] = filter_capitalized_words(empty_df.loc[i, 'obj_one_hop'].split(','), 0)
    empty_df.loc[i, 'obj_new_hop_num'], empty_df.loc[i, 'obj_new_one_hop'] = filter_capitalized_words(empty_df.loc[i, 'obj_new_one_hop'].split(','), 0)
empty_df

,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,sbj_views,wiki_subject
0,40,Kharkiv,Q42308,"KhTZ Stadium,UAKIV,Saltivskyi District,Ihor Te...",10,Warsaw,Q270,"Order of the Cross of Grunwald,Bemowo,Ochota,O...",10,Athens,Q1524,"BN,IX,BK,BM,IB,OA,IO,ZH,BP,IP",10,2582277,Kharkiv
1,58,Northwest Territories,Q2007,"Inuvialuktun,Inuktitut,Mackenzie District,Unga...",10,English,Q1860,"Latin script,British Virgin Islands,United Sta...",10,Tamil,Q5885,"Koleluttu,Mannar District,Sri Potti Sri Ramulu...",10,1660426,Northwest Territories
2,70,Beirut,Q3820,"Beyrouthin,Beyrouthine,LBBEY,Ashrafieh,Beiroet...",10,Istanbul,Q406,"Venice,Shimonoseki,Bosporus Strait,Sea of Marm...",10,Florence,Q2044,"Firenze,Florenz,ITFLR,Grand Duchy of Tuscany,K...",10,3422938,Beirut
3,112,Gwen Stefani,Q483379,"Kingston Rossdale,Apollo Rossdale,Zuma Rossdal...",10,English,Q1860,"British Virgin Islands,United States Virgin Is...",10,French,Q150,"Hainaut,Walloon Brabant,Flemish Brabant,Provin...",10,9055441,Gwen Stefani
4,143,Delusions of Grandeur,Q705247,"Lita Recio,Roger Carel,Ricardo Palacios,Venant...",10,French,Q150,"Hainaut,Walloon Brabant,Flemish Brabant,Romanc...",10,English,Q1860,"Vulgar Latin,Germanic,Received Pronunciation,O...",10,28646,Delusions of Grandeur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,21827,Indonesia,Q252,"North Sulawesi,North Sumatra,West Nusa Tenggar...",10,Indonesian,Q9240,Agency for Language Development and Cultivatio...,10,English,Q1860,"Vulgar Latin,Germanic,Received Pronunciation,O...",10,18163294,Indonesia
996,21836,Gregory XV,Q132692,"Scipione Borghese,Gregorius,Gregor,Latin Catho...",10,pope,Q19546,"Apostolic Palace,College of Cardinals,Diocese ...",10,bishop,Q29182,"Regesta Imperii XIII,Obispo,Excellency,Tamil E...",10,139515,Gregory XV
997,21872,Paul Huet,Q1354027,"Dordrechts Museum,Museum of the French Revolut...",10,Paris,Q90,"Kingdom of France,Puteaux,Seine,Companion of t...",10,Venice,Q641,"Contrada della Chiocciola,Mira,Rimini,Aksaray,...",10,10254,Paul Huet
998,21901,Mar del Plata,Q52607,"Biarritz,Salzano,Sorrento,Vigan,Palma,Fort Lau...",10,Havana,Q1563,"La Habana,Ciutat de les columnes,Habanero,Hava...",10,Tehran,Q3616,"Pahlavi Iran,Teheranano,Qajar Iran,IRTHR,Teher...",10,401838,Mar del Plata


In [ ]:
# 필터링: empty_df의 'index' 값과 df의 'index' 값이 같은 행만 추출
answer = answer[answer['index'].isin(empty_df['index'])]
answer = answer.reset_index(drop = True)

for i in range(len(answer)):
    answer.loc[i, 'sbj_hop_num'], answer.loc[i, 'sbj_one_hop'] = filter_capitalized_words(answer.loc[i, 'sbj_one_hop'].split(','), 0)
    answer.loc[i, 'obj_true_hop_num'], answer.loc[i, 'obj_one_hop'] = filter_capitalized_words(answer.loc[i, 'obj_one_hop'].split(','), 0)
    answer.loc[i, 'obj_new_hop_num'], answer.loc[i, 'obj_new_one_hop'] = filter_capitalized_words(answer.loc[i, 'obj_new_one_hop'].split(','), 0)


answer


,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,sbj_views,wiki_subject
0,40,Kharkiv,Q42308,"KhTZ Stadium,charkiviti,UAKIV,citykharkov,khar...",28,Warsaw,Q270,"Bemowo,Ochota,Otwock County,city with powiat r...",49,Athens,Q1524,"League of Historical Cities,BN,IX,IZ,BK,BM,IB,...",94,2582277,Kharkiv
1,58,Northwest Territories,Q2007,"parliamentary system,Franklin District,Ungava ...",40,English,Q1860,"Malta,Bahrain,Vanuatu,Israel,Sierra Leone,Unit...",101,Tamil,Q5885,"Koleluttu,ta,tam,tsp,tml,Karnataka,Kerala,Andh...",20,1660426,Northwest Territories
2,70,Beirut,Q3820,"Beyrouthin,Beyrouthine,beritensi,LBBEY,beyrout...",32,Istanbul,Q406,"megacity,Bosporus Strait,Mary,Guangzhou,Consta...",60,Florence,Q2044,"Firenze,Florenz,firenze,Italian leaves,florent...",30,3422938,Beirut
3,112,Gwen Stefani,Q483379,"rock music,Kingston Rossdale,Apollo Rossdale,Z...",17,English,Q1860,"Malta,Bahrain,Israel,Sierra Leone,United Arab ...",96,French,Q150,"Province of Namur,present tense,present partic...",43,9055441,Gwen Stefani
4,143,Delusions of Grandeur,Q705247,"video on demand,Gaumont Film Company,Germany,c...",33,French,Q150,"Province of Namur,present tense,present partic...",41,English,Q1860,"Malta,Bahrain,Israel,Sierra Leone,United Arab ...",96,28646,Delusions of Grandeur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,21827,Indonesia,Q252,"North Sulawesi,North Sumatra,West Nusa Tenggar...",838,Indonesian,Q9240,Agency for Language Development and Cultivatio...,14,English,Q1860,"Malta,Bahrain,Vanuatu,Israel,Sierra Leone,Naur...",89,18163294,Indonesia
996,21836,Gregory XV,Q132692,"copyrights on works have expired,Victoria and ...",19,pope,Q19546,"College of Cardinals,history of the papacy,Dio...",23,bishop,Q29182,Bible Encyclopedia of Archimandrite Nicephorus...,15,139515,Gregory XV
997,21872,Paul Huet,Q1354027,"Art Institute of Chicago,Svensk uppslagsbok,Na...",25,Paris,Q90,"megacity,department of France,Pantin,metropoli...",47,Venice,Q641,"Contrada della Chiocciola,commune of Italy,Lis...",58,10254,Paul Huet
998,21901,Mar del Plata,Q52607,"ARMDQ,Palma,Acireale,locality,Argentina,Bari,B...",14,Havana,Q1563,"Armenian Soviet Encyclopedia,La Habana,Ciutat ...",31,Tehran,Q3616,"League of Historical Cities,Pahlavi Iran,Teher...",32,401838,Mar del Plata
